In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from tqdm import tqdm
import transformers
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

from data import *

In [10]:
dataset_path = '/home/shawn/nvme/vl_research/jerry-agent/SlimPajama-6B/data'
train_dataset, validation_dataset, test_dataset = get_slimpajama_dataset(dataset_path)

Loading dataset...


  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:40<00:00,  1.23it/s]


In [11]:
# train_dataset, validation_dataset, test_dataset = get_slimpajama_dataset(dataset_path)

In [12]:
tokenizer = get_tokenizer("lmsys/vicuna-7b-v1.5")

Loading tokenizer...


In [13]:
train_dataloader = SlimPajamaDataset(train_dataset, tokenizer)

In [14]:
validation_dataloader = SlimPajamaDataset(validation_dataset, tokenizer)

In [15]:
# train_dataloader = SlimPajamaDataset(train_dataset, tokenizer)
# validation_dataloader = SlimPajamaDataset(validation_dataset, tokenizer)
test_dataloader = SlimPajamaDataset(test_dataset, tokenizer)

In [16]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

def get_traing_arguments():
    training_args = TrainingArguments(
        output_dir="my_awesome_eli5_clm-model",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        weight_decay=0.01,
        push_to_hub=False,
        num_train_epochs=5
    )
    return training_args

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
import json

from transformers import LlamaConfig, LlamaForCausalLM

with open('configs/config.json', 'r') as file:
    config_dict = json.load(file)

# Create the configuration object
config = LlamaConfig.from_dict(config_dict)

# Initialize the LlamaForCausalLM model
model = LlamaForCausalLM(config)



In [18]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# Huggingface Trainer setup
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=50,  # Log every 50 steps
    report_to="all"  # or use "none" if you don't want to report metrics
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator
)

# Train
trainer.train()



--------Hi, I am in trainer.py constructor--------


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Failed to detect the name of

--------Hi, I am in trainer.py train()--------
check start train
check strarting training loop
check train_dataloader have length: True
check self.state.max_steps: 8043
check self.state.num_train_epochs: 3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Currently logged in as: wel019. Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the env

epoches_trained: 0; num_train_epochs: 3
epoch: 0


AttributeError: 'str' object has no attribute 'size'

In [ ]:
trainer.evaluate()